### Installing and importing required libraries.

In [ ]:
# Mounting files
import os
import pandas as pd
import random
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install transformers
!pip install evaluate
!pip install sentencepiece
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.8 MB/s eta 0:00:00

In [ ]:
import torch
import re
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pytorch_lightning as pl
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download('omw-1.4')
import os
import string
import torch.nn as nn
import pandas as pd
import numpy as np
import evaluate
import matplotlib.pyplot as plt
import transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import get_linear_schedule_with_warmup, AdamW
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
## Load the Excel file into a pandas DataFrame
df = pd.read_excel("/gdrive/MyDrive/datasets/Inshorts_Cleaned_Data.xlsx")

# Keep only 'Headline' and 'Short' columns and discard remaining columns
df = df[['Headline', 'Short']]

# Randomly sample 50000 rows without replacement from the dataframe
df_sample = df.sample(n=50000, replace=False)

# Randomly allocate the 50000 rows into 3 different groups with the specified proportions
train_size = 30000
test_size = 10000
dev_size = 10000

train_set = df_sample.sample(n=train_size, replace=False)
df_sample = df_sample.drop(train_set.index)

test_set = df_sample.sample(n=test_size, replace=False)
dev_set = df_sample.drop(test_set.index)

# Export each group to a separate Excel file
train_set.to_excel('/gdrive/MyDrive/datasets/pr-train-large.xlsx', index=False)
test_set.to_excel('/gdrive/MyDrive/datasets/pr-test-large.xlsx', index=False)
dev_set.to_excel('/gdrive/MyDrive/datasets/pr-dev-large.xlsx', index=False)

In [ ]:
# Read smaller sample datasets as Pandas dataframes
data_train = pd.read_excel('/gdrive/MyDrive/datasets/pr-train-large.xlsx')
data_train.head()

,Headline,Short
0,Computer beats man in Chinese game for 1st time,In a major breakthrough for artificial intelli...
1,"‘Triple talaq is valid, but its misuse shouldn...",The imams from National Capital Region on Satu...
2,Heat from Jupiter moon can support ocean: Study,"According to a new study, Jupiter’s moon Europ..."
3,Omar-led delegation meets Prez over Kashmir,A delegation of opposition leaders from J&amp;...
4,"Arpita shares pic of Salman, Ahil and Aayush",Arpita Khan Sharma has shared a picture on Ins...


In [ ]:
data_test = pd.read_excel('/gdrive/MyDrive/datasets/pr-test-large.xlsx')
data_test.head()

,Headline,Short
0,&#39;Indian Gaming League&#39; launched in India,India&#39;s first professional e-sports league...
1,"RGV calls Tiger a &#39;bikini babe&#39;, compa...","Filmmaker Ram Gopal Varma, sharing Tiger Shrof..."
2,Zakir Naik’s NGO gave ₹50L to Rajiv Gandhi trust,Televangelist Zakir Naik’s NGO Islamic Researc...
3,Knew nothing about tax fraud: Messi,Barcelona forward Lionel Messi on Thursday sai...
4,Rajasthan High Court scraps Gujjar reservation,The Rajasthan High Court on Friday quashed the...


In [ ]:
data_validate = pd.read_excel('/gdrive/MyDrive/datasets/pr-dev-large.xlsx')
data_validate.head()

,Headline,Short
0,Samsung admits waterproofing defect in S7 Active,Following a report by consumer-advocacy group ...
1,SC summons Katju to debate Soumya case verdict,Following his criticism of the Soumya rape cas...
2,Device to power mobile through swipes developed,Michigan State University researchers have dev...
3,India gives El Nino updates to S Asian nations,With information updated by the India Meteorol...
4,"Real draw Bayern, Barcelona get Juventus in CL...",Defending champions Real Madrid will play Baye...


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

def preprocess(text):
    '''
    Returns a cleaned string called 'text'
    '''
    text = text.lower() # lowercase
    # convert have'nt -> have not
    text = text.split()
    for i in range(len(text)):
        word = text[i]
        if word in contraction_mapping:
            text[i] = contraction_mapping[word]
    # join entire text after contraction mapping is completed
    text = " ".join(text)
    text = text.split()
    newtext = []
    # create list of words after removing stopwords
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    # create string of same words
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words inside a parenthesis)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    return text

sample = "(hello) hi there .man tiger caller who's that isn't it ? WALL-E"
print(preprocess(sample))

 hi  . man tiger caller  walle


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Preprocess training data
data_train['Headline'] = data_train['Headline'].apply(lambda x:preprocess(x))
data_train['Short'] = data_train['Short'].apply(lambda x:preprocess(x))


# Preprocess test data
data_test['Headline'] = data_test['Headline'].apply(lambda x:preprocess(x))
data_test['Short'] = data_test['Short'].apply(lambda x:preprocess(x))


# Preprocess validation data
data_validate['Headline'] = data_validate['Headline'].apply(lambda x:preprocess(x))
data_validate['Short'] = data_validate['Short'].apply(lambda x:preprocess(x))


In [ ]:
data_train['Short'][:5]

0    major breakthrough artificial intelligence com...
1    imams national capital region saturday said pr...
2    according new study jupiters moon europa creat...
3    delegation opposition leaders jampk led excm o...
4    arpita khan sharma shared picture instagram br...
Name: Short, dtype: object

### Model hyperparameters setting
T5 comes in 5 different sizes:
- t5-small: 60M parameters
- t5-base: 220M parameters
- t5-large: 770M parameters
- t5–3b: 3B parameters
- t5–11b: 11B parameters

All five variants support English, French, Romanian, and German languages.
The below code shows the hyperparameter setting.

In [ ]:
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
BATCH_SIZE = 4
TEXT_LEN = 512
HEADLINE_LEN = 64
EPOCHS = 5

In [ ]:
# CUDA
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Add “Summarize: “ at the beginning of each instance (i.e., every unique source text), as T5 requires a task-specific prefix at the start of every instance or sentence.
I have used a text length of 512 and a summary length of 64.
The class in turn returns,
- input_ids: tokenized input ids of length 512 token_ids for input text.
- summary_ids: tokenized summary ids of length 64 token_ids for a summary.
- Attention masks for both.

In [ ]:
class NewsSummaryDataset(Dataset):
    def __init__(self, df, tokenizer, text_len, headline_len):
        self.df = df
        self.headlines = self.df["Headline"]
        self.text = self.df["Short"]
        self.tokenizer = tokenizer
        self.text_len = text_len
        self.headline_len = headline_len
  
    def __len__(self):
        return len(self.headlines)
  
    def __getitem__(self, idx):
        # T5 transformers performs different tasks by prepending the particular prefix to the input text.
        text = "summarize:" + str(self.text[idx])                # In order to avoid dtype mismatch, as T5 is text-to-text transformer, the datatype must be string
        headline = str(self.headlines[idx])

        text_tokenizer = self.tokenizer(text, max_length=self.text_len, padding="max_length",
                                                        truncation=True, add_special_tokens=True)
        headline_tokenizer = self.tokenizer(headline, max_length=self.headline_len, padding="max_length",
                                                        truncation=True, add_special_tokens=True)
        return {
            "input_ids": torch.tensor(text_tokenizer["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(text_tokenizer["attention_mask"], dtype=torch.long),
            "summary_ids": torch.tensor(headline_tokenizer["input_ids"], dtype=torch.long),
            "summary_mask": torch.tensor(headline_tokenizer["attention_mask"], dtype=torch.long)
        }
     

### PyTorch Lightning LightningDataModule
A data module is a collection of data loaders along with required transforms and data processing pipelines.
PyTorch Lightning’s data module has 5 steps involved in the processing of data
1. Tokenizing or processing.
2. Data cleaning.
3. Load the dataset.
4. Data transformation (rotate, tokenize, etc…).
5. Wrapping the dataset into data loaders.

Data settings:
1. Batch size: 4
2. num_workers: 4 (the number of processes that generate batches in parallel).
3. Splitting size: 80% (training) and 20%(testing).

In [ ]:
class NewsSummaryDataModule(pl.LightningDataModule):
    def __init__(self, 
                train_df, 
                val_df,
                test_df,
                batch_size,
                tokenizer,
                text_len,
                summary_len):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.batch_size = batch_size
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.text_len = text_len
        self.summary_len = summary_len
    
    def setup(self, stage=None):
        self.train_dataset = NewsSummaryDataset(
            self.train_df,
            self.tokenizer,
            self.text_len,
            self.summary_len)
        
        self.val_dataset = NewsSummaryDataset(
            self.val_df,
            self.tokenizer,
            self.text_len,
            self.summary_len)
        
        self.test_dataset = NewsSummaryDataset(
            self.test_df,
            self.tokenizer,
            self.text_len,
            self.summary_len
        )
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=4)
        
    def test_dataloader(self):
        return DataLoader(
              self.test_dataset,
              batch_size=self.batch_size,
              num_workers=4
          )
     

### Calling the above-defined classes and load the data loaders.

In [ ]:
news_summary_module = NewsSummaryDataModule(data_train, data_validate, data_test, BATCH_SIZE, TOKENIZER, TEXT_LEN, HEADLINE_LEN)
news_summary_module.setup()

In [ ]:
next(iter(news_summary_module.train_dataloader().dataset))

{'input_ids': tensor([21603,    10, 16547,   127, 20974,  7353,  6123,  1218,   478,   166,
            97, 17025,   771,  1959,  4913,     3,  1436,  1496,    15,  1476,
           467,   281,     3,     5,  1597, 10283,  9160,   349,  1659, 14481,
           491, 25173,    32, 17025,   386,   715, 14864,  6336,  1819,     3,
          3464,   943,     3,     5,  2283, 19095,  1218,  1356,     3,   179,
           577,   593,   936, 13217,     7,     3,     5,     1,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

### Building Model with PyTorch Lightning’s LightningModule
LightningModule simplifies PyTorch training and testing loops. Basically, it organizes the code into 6 parts.
1. init: Defines the computations.
2. forward: For inference only.
3. training_step: the entire training loop.
4. validation_step: the entire validation loop.
5. test_step: the entire testing loop.
6. predict_step: the prediction loop.
7. configure_optimizers: For configuring optimizers and schedulers.

In [ ]:
class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super(NewsSummaryModel, self).__init__()
        self.model = MODEL
        self.model = self.model.to(DEVICE)
  
    def forward(self, input_ids, attention_mask, labels=None, decoder_attention_mask=None):
        outputs = self.model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels,
                        decoder_attention_mask=decoder_attention_mask)
        return outputs.loss, outputs.logits
  
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["summary_ids"]
        decoder_attention_mask = batch["summary_mask"]

        loss, output = self(input_ids, attention_mask, labels, decoder_attention_mask)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["summary_ids"]
        decoder_attention_mask = batch["summary_mask"]

        loss, output = self(input_ids, attention_mask, labels, decoder_attention_mask)
        return loss
 
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        loss, output = self(input_ids=input_ids, 
                      attention_mask=attention_mask)
        return loss
    
    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=0.0001)
        scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=0,
                num_training_steps=EPOCHS*len(df))
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

In [ ]:
model = NewsSummaryModel()
model = model.to(DEVICE)

### Training the model with Lightning Trainer
After organizing the code in the above class and inheriting LightningModule, the Lightning Trainer does the rest. It manages data loaders, puts batches onto the same device, and calls callbacks when necessary.
I have set max_epochs to 5. Just call the fit method once all the settings have been made.

In [ ]:
trainer = pl.Trainer(
    max_epochs=EPOCHS
    )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, news_summary_module)

### Load the best model from the saved checkpoints. 
Since we are using the model for inference, we need to call the freeze() method to freeze all parameters.

In [ ]:
# Loading the best model

news_summary_model = NewsSummaryModel.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
news_summary_model.freeze()

We have written a small inference code that,
- Takes raw text as input.
- The tokenizer produces input_ids and attention_masks from it.
- The tokenizer outputs are fed to the model’s generate method, which in return outputs the token_ids for the predicted summary.
- Then just decode the token_ids and get the text output.

In [ ]:
def summarize(text):
    inputs = TOKENIZER(text, 
                       max_length=TEXT_LEN,
                       truncation=True,
                       padding="max_length",
                       add_special_tokens=True, 
                       return_tensors="pt")
    inputs = {key: tensor.to(DEVICE) for key, tensor in inputs.items()}
    summarized_ids = news_summary_model.model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"], 
        num_beams=4)

    return " ".join([TOKENIZER.decode(token_ids, skip_special_tokens=True)
                    for token_ids in summarized_ids])

### BLEU metric
In addition to the inference code, we used the BLEU metric on the Test dataset

In [ ]:
bleu = evaluate.load("google_bleu")
total_bleu_score = 0
for i in range(100):
    text = data_test.iloc[i]["Short"]
    headline = data_test.iloc[i]["Headline"]
    summarized_text = summarize(text)
    total_bleu_score = total_bleu_score + bleu.compute(predictions=[summarized_text], references=[headline])['google_bleu']
avg_bleu_score = total_bleu_score/100
print(avg_bleu_score)

0.26407661000849864


In [ ]:
import pickle
filename = open('/gdrive/MyDrive/AML/FinalProject/Best_T5_Model_big_2.pkl', 'wb')
pickle.dump(news_summary_model.model, filename)

In [ ]:
for i in range(100):
    text = data_test.iloc[i]["Short"]
    headline = data_test.iloc[i]["Headline"]
    summarized_text = summarize(text)
    print("Actual: ", headline)
    print("Predicted: ", summarized_text)

Actual:  39indian gaming league39 launched india
Predicted:  esports league igl launched india february 4
Actual:  rgv calls tiger 39bikini babe39 compares urmila
Predicted:  bruce lee wouldn39t hav bcm bruce lee
Actual:  zakir naiks ngo gave 50l rajiv gandhi trust
Predicted:  zakir naiks ngo irf donated 50l
Actual:  knew nothing tax fraud messi
Predicted:  messi knew nothing tax evasion messi
Actual:  rajasthan high court scraps gujjar reservation
Predicted:  rajasthan hc quakes 5 reservation special backward classes
Actual:  kejriwal compares power tariffs delhi gujarat
Predicted:  pm modi wants delhi power tariffs gujarat kejr
Actual:  want make india open economy world pm modi
Predicted:  want make india open economy world modi
Actual:  2 crore people attend msg lion heart 2 trailer launch
Predicted:  trailer gurmeet39s 39hind ka napak ko jaw
Actual:  twice many army men killed jampk 2016 2014 2015
Predicted:  twice many indian soldiers martyred year jampk
Actual:  asus launch curv